<a href="https://colab.research.google.com/github/zobayer-al-mahmud/Sentiment-Analysis-on-Product-Reviews-IMDB-/blob/main/Module11_IMDB_Sentiment_%3CZobayer_Al_Mahmud%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 66.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# **Dataset Load**

In [ ]:
imdb = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
imdb['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
train_df.head()

,text,label
6868,"Dumb is as dumb does, in this thoroughly unint...",0
24016,I dug out from my garage some old musicals and...,1
9668,After watching this movie I was honestly disap...,0
13640,This movie was nominated for best picture but ...,1
14018,Just like Al Gore shook us up with his painful...,1


# **Preprocessing**


In [ ]:
import re
import string

def clean_text(text):
    text = text.lower() # Lowercasing
    text = re.sub('<.*?>', '', text) # Remove HTML tags
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    return text

train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_text(text):
    return tokenizer(text, padding=True, truncation=True, return_tensors="pt")

train_encodings = train_df['cleaned_text'].apply(tokenize_text)
test_encodings = test_df['cleaned_text'].apply(tokenize_text)

print("Preprocessing complete.")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Preprocessing complete.


In [ ]:
print("Quantity of cleaned text in training data:", len(train_df['cleaned_text']))
print("Quantity of cleaned text in test data:", len(test_df['cleaned_text']))

Quantity of cleaned text in training data: 2000
Quantity of cleaned text in test data: 1000


# **Vectorization, Model Training & Evaluation**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # You can adjust max_features
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['cleaned_text'])

y_train = train_df['label']
y_test = test_df['label']

# Train a Logistic Regression Classifier
tfidf_model = LogisticRegression(max_iter=1000) # Increase max_iter if convergence warnings occur
tfidf_model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test, y_pred_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

print(f"TF-IDF Logistic Regression Precision: {precision_tfidf}")
print(f"TF-IDF Logistic Regression Recall: {recall_tfidf}")
print(f"TF-IDF Logistic Regression F1 Score: {f1_tfidf}")
print(f"TF-IDF Logistic Regression Accuracy: {accuracy_tfidf}")

TF-IDF Logistic Regression Precision: 0.8108651911468813
TF-IDF Logistic Regression Recall: 0.8241308793456033
TF-IDF Logistic Regression F1 Score: 0.8174442190669371
TF-IDF Logistic Regression Accuracy: 0.82


In [ ]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# ===============================
# Tokenize the cleaned text into words for Word2Vec
# ===============================
tokenized_text_train = [text.split() for text in train_df['cleaned_text']]
tokenized_text_test = [text.split() for text in test_df['cleaned_text']]

# ===============================
# Train Improved Word2Vec Model (Option 2)
# ===============================
# Updated parameters: larger vector size, skip-gram, min_count=5, more epochs
word2vec_model = Word2Vec(
    sentences=tokenized_text_train,
    vector_size=300,
    window=5,
    min_count=5,
    sg=1,
    workers=4,
    epochs=10
)

# ===============================
# Function to get the average Word2Vec vector for a document
# ===============================
def document_vector(doc, model):
    words = [word for word in doc if word in model.wv.index_to_key]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

# ===============================
# Generate document vectors for training and testing data
# ===============================
X_train_word2vec = np.array([document_vector(doc, word2vec_model) for doc in tokenized_text_train])
X_test_word2vec = np.array([document_vector(doc, word2vec_model) for doc in tokenized_text_test])

# ===============================
# Train a Logistic Regression Classifier on Word2Vec embeddings
# ===============================
word2vec_model_lr = LogisticRegression(max_iter=300)
word2vec_model_lr.fit(X_train_word2vec, y_train)

# ===============================
# Evaluate the model
# ===============================
y_pred_word2vec = word2vec_model_lr.predict(X_test_word2vec)

accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
precision_word2vec = precision_score(y_test, y_pred_word2vec)
recall_word2vec = recall_score(y_test, y_pred_word2vec)
f1_word2vec = f1_score(y_test, y_pred_word2vec)

print(f"Word2Vec Logistic Regression Precision: {precision_word2vec}")
print(f"Word2Vec Logistic Regression Recall: {recall_word2vec}")
print(f"Word2Vec Logistic Regression F1 Score: {f1_word2vec}")
print(f"Word2Vec Logistic Regression Accuracy: {accuracy_word2vec}")


Word2Vec Logistic Regression Precision: 0.7681451612903226
Word2Vec Logistic Regression Recall: 0.7791411042944786
Word2Vec Logistic Regression F1 Score: 0.7736040609137056
Word2Vec Logistic Regression Accuracy: 0.777


In [ ]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

# ===============================
# Load model and tokenizer
# ===============================
# Load the pre-trained DistilBERT tokenizer and model for sequence classification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2) # num_labels=2 for binary classification (positive/negative)

# Determine the device to use (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # Move the model to the selected device

# ===============================
# Tokenize and prepare data
# ===============================
# Tokenize the cleaned text data from the training and test dataframes
# truncation=True: Truncate sequences longer than max_length
# padding=True: Pad sequences shorter than max_length
# max_length=128: Maximum sequence length
# return_tensors='pt': Return PyTorch tensors
train_encodings = tokenizer(list(train_df['cleaned_text']), truncation=True, padding=True, max_length=256, return_tensors='pt')
test_encodings  = tokenizer(list(test_df['cleaned_text']),  truncation=True, padding=True, max_length=256, return_tensors='pt')

# Convert labels to PyTorch tensors
train_labels_tensor = torch.tensor(y_train)
test_labels_tensor  = torch.tensor(y_test)

# Create PyTorch TensorDatasets from the tokenized data and labels
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels_tensor)
test_dataset  = TensorDataset(test_encodings['input_ids'],  test_encodings['attention_mask'],  test_labels_tensor)

# Create PyTorch DataLoaders for efficient batching of data during training and evaluation
train_loader  = DataLoader(train_dataset, batch_size=8, shuffle=True) # Smaller batch size for fine-tuning
test_loader   = DataLoader(test_dataset,  batch_size=8)

# ===============================
# Optimizer
# ===============================
# Initialize the AdamW optimizer for fine-tuning the model
optimizer = AdamW(model.parameters(), lr=2e-5) # Learning rate is often small for fine-tuning

# ===============================
# Fine-tuning Loop
# ===============================
epochs = 10 # Number of training epochs
model.train()  # Set the model to training mode

print(f"Starting fine-tuning for {epochs} epochs...")
for epoch in range(epochs):
    total_loss = 0 # Initialize total loss for the epoch
    # Iterate over batches in the training data loader
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # Move batch tensors to the selected device
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad() # Clear previous gradients

        # Perform a forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss # Get the loss from the model outputs

        loss.backward() # Perform backpropagation to calculate gradients
        optimizer.step() # Update model parameters

        total_loss += loss.item() # Accumulate the batch loss

    # Calculate and print the average loss for the epoch
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} average loss: {avg_train_loss:.4f}")

# ===============================
# Evaluation
# ===============================
model.eval() # Set the model to evaluation mode
preds, true_labels = [], [] # Lists to store predictions and true labels

print("\nEvaluating the model...")
# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over batches in the test data loader
    for batch in tqdm(test_loader, desc="Evaluating"):
        # Move batch tensors to the selected device
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        # Perform a forward pass to get logits (raw model outputs before activation)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get the predicted labels by finding the index of the maximum logit
        preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
        # Store the true labels
        true_labels.extend(labels.cpu().numpy())

# ===============================
# Compute Metrics
# ===============================
# Calculate evaluation metrics using scikit-learn
accuracy_bert  = accuracy_score(true_labels, preds)
precision_bert = precision_score(true_labels, preds)
recall_bert    = recall_score(true_labels, preds)
f1_bert        = f1_score(true_labels, preds)

# Print the evaluation results
print(f"BERT Embeddings Logistic Regression Precision: {precision_bert}")
print(f"BERT Embeddings Logistic Regression Recall: {recall_bert}")
print(f"BERT Embeddings Logistic Regression F1 Score: {f1_bert}")
print(f"BERT Embeddings Logistic Regression Accuracy: {accuracy_bert}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fine-tuning for 10 epochs...


Epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 average loss: 0.4353


Epoch 2:   0%|          | 0/250 [00:00<?, ?it/s]

...

Epoch 9 average loss: 0.0248


Epoch 10:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 10 average loss: 0.0059

Evaluating the model...


Evaluating:   0%|          | 0/125 [00:00<?, ?it/s]

BERT Embeddings Logistic Regression Precision: 0.8670756646216768
BERT Embeddings Logistic Regression Recall: 0.8670756646216768
BERT Embeddings Logistic Regression F1 Score: 0.8670756646216768
BERT Embeddings Logistic Regression Accuracy: 0.87


# **Comparison & Analysis**

In [ ]:
import pandas as pd

# Create a dictionary to store the metrics for each model
results = {
    'Model': ['TF-IDF', 'Word2Vec', 'BERT Embeddings'],
    'Accuracy': [accuracy_tfidf, accuracy_word2vec, accuracy_bert],
    'Precision': [precision_tfidf, precision_word2vec, precision_bert],
    'Recall': [recall_tfidf, recall_word2vec, recall_bert],
    'F1 Score': [f1_tfidf, f1_word2vec, f1_bert]
}

# Create a pandas DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Display the DataFrame
display(results_df)

,Model,Accuracy,Precision,Recall,F1 Score
0,TF-IDF,0.820,0.810865,0.824131,0.817444
1,Word2Vec,0.777,0.768145,0.779141,0.773604
2,BERT Embeddings,0.870,0.867076,0.867076,0.867076


Here is a short analysis of the model performance:

*   **Best Performing Approach:** Based on the accuracy and F1-score in the comparison table, the **BERT Embeddings** approach worked best. It achieved the highest values across all presented metrics (Accuracy, Precision, Recall, and F1 Score).
*   **Why BERT Embeddings Worked Best:** BERT, being a pre-trained transformer model, is capable of capturing more complex contextual relationships and nuances in the text compared to TF-IDF and Word2Vec. This ability to understand context often leads to better performance on downstream tasks like sentiment analysis, especially when fine-tuned on the specific task data.
*   **Trade-offs:**
    *   **Accuracy:** BERT Embeddings provided the highest accuracy, followed by TF-IDF, and then Word2Vec.
    *   **Training Time:** Training the BERT model and generating embeddings is significantly more time-consuming and computationally intensive compared to TF-IDF and Word2Vec. TF-IDF is generally the fastest to train.
    *   **Resource Usage:** BERT requires substantially more memory and computational resources (ideally a GPU) than TF-IDF and Word2Vec, which can be trained on standard CPUs with less memory.
*   **Observations about Common Errors:**
    *   **Data Preprocessing:** While basic cleaning was performed, more advanced techniques like stemming or lemmatization could be explored, although their impact varies.
    *   **Hyperparameter Tuning:** The performance of all models, particularly Word2Vec and BERT, is sensitive to hyperparameters. Optimal parameters were not extensively searched in this example.
    *   **Dataset Size:** The dataset size (2000 training examples) might be relatively small for fully leveraging the power of large models like BERT. Performance might improve with more data.
    *   **Model Complexity vs. Data:** For smaller datasets, simpler models like Logistic Regression with TF-IDF can sometimes perform competitively with less computational cost.
    *   **BERT Fine-tuning:** The fine-tuning process for BERT requires careful consideration of learning rates, epochs, and batch sizes to avoid overfitting or underfitting.